# Plan of the workshop

## I Optimization
- Optimization in a Nutshell
- JuliaNLSolvers: Optim.jl
- Optim.jl: syntax and examples
- LsqFit.jl: optimization and curve fitting
- Outside of JNLS

## II Solving (systems of) equations
- The problem
- JuliaNLSolvers: NLsolve.jl

# Optimization

# Optimization in a nutshell
Optimization is a lot of things, but for the purpose of this workshop:

$$
\begin{align}
&\text{minimize }&&f_0(x)\\
   &\text{subject to }~&&f_i(x) \leq 0,~i = 1..n\\
    &{} &&h_j(x) =  0,~j = 1..m
    \end{align}
$$

Where $f_0$ is the objective, $f_i$'s for $i>0$ are the inequality constraints and $h_i$'s are the equality constrains.

Topics: unconstrained, convex (linear, quadratic, ...), polynomial, and so on.

Many classes of problems exist, but today we will look at what is often called nonlinear programming or optimization, even if many other classes are also nonlinear.

## Unconstrained Optimization
Unconstrained optimization refers to the case with no $f_i$ or $h_i$ constraints.

Now, what is our job then? Find an $x$ such that for all $x'
\in\mathbf{dom }~f$ we have that

$$
f(x)\leq f(x')
$$

$x$ is then called a global minimizer and $f(x)$ is called the global minimum.

## Example: Unconstrained Optimization
Unconstrained optimization refers to the case where $f_i(x)=0$ for all $i\in\{1,\ldots,n\}$ and $h_i(x)=0$ for all $j\in\{1\ldots,m\}$.

Now, what is our job then? Find an $x$ such that for all $x'
\in\mathbf{dom } f$ we have that

$$
f(x)\leq f(x')
$$

Of course, this is trivial here, because $f(x)$ is convex, so if we find and $x$ such that the condition holds only locally, then we've found the global solution as well.

## Univariate case

### Quadratic


$$
\begin{align}
f_0(x)=f(x)=x^2
\end{align}
$$
![using Plots, LaTeXStrings
gr()
plot(x->x^2; legend=false)
savefig("quadratic.png")](https://juliabox.com/notebooks/quadratic.png)

Minima, global minimum, minimizers?

### A trigonometric function

$$
f(x) = sin(x)
$$

![plot((-3π:0.01:3π),x->sin(x);legend=false)
savefig("sin.png")](https://juliabox.com/notebooks/sin.png)

Minima, global minimum, minimizers?

### Wave, wave, wave, ...
$$
f(x) = sin(x)/x
$$
![plot((-20π:0.01:20π),x->sin(x)/x;legend=false)
savefig("sinoverx.png")](https://juliabox.com/notebooks/sinoverx.png)

## Multivariate
Moving to multivariate function, we could consider the so called Rosenbrock test function.
$$
f({\mathbf  {x}})=f(x_{1},x_{2},\dots ,x_{N})=\sum _{{i=1}}^{{N/2}}\left[100(x_{{2i-1}}^{2}-x_{{2i}})^{2}+(x_{{2i-1}}-1)^{2}\right]
$$
or in two dimensions
$$
f({\mathbf  {x}})=f(x_{1},x_{2})= (1-x_1^2+100(x_2-x_1^2))^2
$$
or in code

In [1]:
rosenbrock(x) =  (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2

rosenbrock (generic function with 1 method)

![contour(-1.5:0.01:2, -.50:0.01:2, (x,y)->sqrt(f([x, y])), fill=false, color=:deep, legend=false, levels=100)
  ](banana.png)

## JuliaNLSolvers intro I: Scope of Optim
Basically, we support

- Derivative-free (heuristics)
- First order (uses the gradient)
- Second order (uses the gradient *and* Hessian)

for multivariate unconstrained or box constrained minimization. For constrained univariate optimization, we also have two derivative free methods.

### Derivative-free methods
These methods are all heuristics that might do a pretty good job at finding a point $x$ that makes $f(x)$ small, using only $f$ and some set of rules for trying different $x$'s

### First order methods
These use the gradient to find a search direction from a given iterate, and possibly given the history of iterates. For differentiable problems, they can be a lot faster and more accurate than the above.

### Second order methods
These use the Hessian in a addition to the gradient to exploit local curvature when choosing a search direction. If the Hessian is available, and is not too large, these methods are very powerful.

### Case study: Nelder-Mead

Maintain a simplex (n-d "triangle" of points). Taking a step then amounts to trying to replace the worst point with a new point by: reflecting, expanding, contracting, or shrinking.

### Case study: Gradient Descent
Probably one of the simplest approaches to minimizing a differentiable function is to use gradient descent. The idea here is simply to choose a search direction
$$
s^k = -\nabla f(x^k)
$$
with a final step of
$$
x^{k+1} = x^k+\alpha s
$$
![source: https://en.wikipedia.org/wiki/Gradient_descent#/media/File:Gradient_descent.svg](https://juliabox.com/notebooks/gradientdescent.png)

### Case study: Newton's method
The Holy Grail, the Gold Standard, and what all other methods want to be: Newton's method. We incorporate the curvature by replacing the gradient descent steps with

$$
    s = -H(x^k)^{-1}\nabla f(x^k)
$$

and then 

$$
    x^{k+1} = x^k+\alpha s
$$

### Interpretation: (Quasi-)Newton solvers 1/2
Above we saw a univariate quadratic function. Consider instead the multivariate quadratic problem

$$
\text{minimize}~r+q^\top x+(1/2)x^\top P x
$$

the so-called first order conditions are obtained by calculating the gradient $\nabla f(x)$ of the function, and solving the problem $\nabla f(x) = 0$. If $P\succ 0$, we can solve this problem by calculating
$$
x_{opt} = -P^{-1}q
$$

### Interpretation: (Quasi-)Newton solvers 2/2
the fact that we can solve the first order conditions **analytically** is a very big deal, and can be seen as the basis for all the methods we might call quasi-Newton methods. They do so by forming quadratic function approximations to our function $f(x)$ around a point $x_0$

$$
\text{minimize}~\tilde{f}(x_0+s) = f(x_0) + \nabla f(x_0)^\top s + (1/2)s^\top P s
$$

with solution
$$
s = -P^{-1}\nabla f(x_0)
$$
We call this the search direction. 

We introduce a step-length $\alpha$, such that the final updating rule for $x$ is
$$
x_{next} = x_{current} +\alpha s
$$

### Set up the package
To make sure everything is working and is up to date (if you dare), run.


In [2]:
#Pkg.add("Optim")
#Pkg.update() # I'm too scared to break anything
using Optim

### Syntax
To optimize (minimize) a function, you have to provide, *at the very least*, a function to calculate the objective and a starting point.
```julia
optimize(objective args..., initial_x, [method], [options])
```
some examples...

In [27]:
obj(x) = sum(x->x^2, x)
x0 = rand(30)
optimize(obj, x0)
# optimize(obj, x0, NelderMead(), Optim.Options())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.6068708819475637,0.648759120319617, ...]
 * Minimizer: [0.023966636355880228,0.10854307566239736, ...]
 * Minimum: 9.465016e-01
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 1282

that didn't go too well... let's try to add gradient information to the mix

In [4]:
obj(x) = sum(x->x^2, x)
g!(g, x) = copy!(g, 2.0*x)
x0 = rand(30)
optimize(obj, g!, x0, GradientDescent()) # Static?

Results of Optimization Algorithm
 * Algorithm: Gradient Descent
 * Starting Point: [0.7096905321603564,0.14128833583107459, ...]
 * Minimizer: [0.0,0.0, ...]
 * Minimum: 0.000000e+00
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 9.95e-01 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 0.00e+00 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 3
 * Gradient Calls: 3

That did the trick! One iteration when using a line search algorithm to find $\alpha$.

Remember the Rosenbrock function from earlier. We have access to the objective, gradient and Hessian from the collection of unconstrained problems in Optim.

In [28]:
prob = Optim.UnconstrainedProblems.examples["Rosenbrock"]
rosf = prob.f
rosg! = prob.g!
rosh! = prob.h!

rosenbrock_hessian! (generic function with 1 method)

Then let's try the wonderful GradientDescent method from earlier... one iteration was needed last time, soo...

In [6]:
optimize(rosf, rosg!, [-3.0, -4.0], GradientDescent())

  0.051878 seconds (29.63 k allocations: 1.721 MiB)


In [7]:
optimize(rosf, rosg!, [-1.0, 1.0], GradientDescent())

  0.000134 seconds (122 allocations: 5.563 KiB)


Results of Optimization Algorithm
 * Algorithm: Gradient Descent
 * Starting Point: [-1.0,1.0]
 * Minimizer: [0.0,0.0]
 * Minimum: 1.000000e+00
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 1.00e+00 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 0.00e+00 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 3
 * Gradient Calls: 3




**So sad...**
what about something that doesn't even use the local information?

In [8]:
optimize(rosf, [-3.0, -4.0], NelderMead())

  0.076020 seconds (62.09 k allocations: 3.275 MiB)


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-3.0,-4.0]
 * Minimizer: [1.000040903622341,1.000087538623777]
 * Minimum: 4.956059e-09
 * Iterations: 95
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 183

That helped... What about using more information?

In [29]:
optimize(rosf, rosg!, rosh!, [-3.0, -4.0], Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [-3.0,-4.0]
 * Minimizer: [0.9999999999445034,0.9999999998967297]
 * Minimum: 9.044243e-21
 * Iterations: 29
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 2.84e-05 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 3.20e-09 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 84
 * Gradient Calls: 84
 * Hessian Calls: 29

Choice of method matters!

## Conditioning and valleys
goto: other notebook

### A knife edge case
$$
f(x) = \frac{1}{2} x_1^2 + \frac{1}{4}x_1^4-\frac{1}{2}x_1^2 \Rightarrow \nabla f(x) = \binom{x_1}{x_2(x_2^2-1)}
$$
start at $x_0=(x_{1,0},x_{2,0})=(\bar{x}, 0)$ for $x_0\in\text{dom }f$

How will Gradient Descent fare? Newton? BFGS?

In [30]:
using Plots
contour(-2:0.01:2, -2:0.01:2, (x,y)->x^2/2+y^4/4-y^2/2, color=:deep,  fill=true,legend=false, levels=50, xlabel="x", ylabel="y")

### A knife edge case

Lesson? Don't blame Optim!

## Missing gradients or Hessians
... but differentiable problem! You can prove it, but you just don't have the patience or skills (but really patience) to calculate it.

Use finite differences from Calculus.jl or automatic differentiation ForwardDiff.jl. 

In [132]:
optimize(rosf, rosg!, rosh!, [-1.0,-1.0], Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [-1.0,-1.0]
 * Minimizer: [0.9999999999990292,0.9999999999980881]
 * Minimum: 1.030282e-24
 * Iterations: 21
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.04e-07 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 1.38e-11 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 62
 * Gradient Calls: 62
 * Hessian Calls: 21

In [133]:
td = TwiceDifferentiable(rosf, [-1.0, -1.0], autodiff = :forward)
# optimize(td, [-1.0,-1.0], Newton())
optimize(td, Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [0.9999999999990292,0.9999999999980881]
 * Minimizer: [0.9999999999990292,0.9999999999980881]
 * Minimum: 1.030282e-24
 * Iterations: 21
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.04e-07 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 1.38e-11 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 62
 * Gradient Calls: 62
 * Hessian Calls: 21

## Options

There are two types of options to set in Optim. Either directly related to the solver or more generally for the behaviour of `optimize`

Options for solvers control tuning parameters and choice of line search

In [ ]:
method = Newton(linesearch=LineSearches.MoreThuente())

Choice of line searches in LineSearches are
 - `MoreThuente`
 - `HagerZhang`
 - `Backtracking` (quadratic or cubic depending on `order=[2|3]` keyword)
 - `StrongWolfe`
 - `Static` (think learning parameter)

Options more generally for `optimize` are set by passing `Optim.Options()` as the last argument to `optimize`

In [32]:
obj(x) = sum(x->x^2, x)
x0 = rand(30)
optimize(obj, x0, Optim.Options(iterations=1000))

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.43607567010310255,0.029047300932380615, ...]
 * Minimizer: [0.04600820798342606,0.2987187609896623, ...]
 * Minimum: 8.580353e-01
 * Iterations: 1000
 * Convergence: false
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 1239

## Traces

To analyze the path from the starting point to the final iterate we can look at the trace. There are three variants here:

- store_trace
- show_trace
- extended_trace

The last one only has an effect if at least one of the two first are `true`.

In [135]:
prob = Optim.UnconstrainedProblems.examples["Hosaki"]
optimize(prob.f, prob.initial_x, Optim.Options(show_trace=true))

Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0    -2.134718e+00     2.877686e+00
     1    -2.134718e+00     4.391230e-01
     2    -2.134718e+00     1.897094e-01
     3    -2.302859e+00     7.395993e-02
     4    -2.302859e+00     1.505622e-02
     5    -2.302859e+00     1.663197e-02
     6    -2.317432e+00     1.138520e-02
     7    -2.330737e+00     8.651347e-03
     8    -2.338368e+00     3.126527e-03
     9    -2.338368e+00     3.739106e-03
    10    -2.344152e+00     2.435949e-03
    11    -2.344152e+00     6.710247e-04
    12    -2.344152e+00     5.175781e-04
    13    -2.344837e+00     4.591353e-04
    14    -2.345267e+00     3.247266e-04
    15    -2.345632e+00     1.716359e-04
    16    -2.345632e+00     3.080862e-05
    17    -2.345697e+00     4.404330e-05
    18    -2.345739e+00     3.383540e-05
    19    -2.345780e+00     2.220605e-05
    20    -2.345791e+00     6.347471e-06
    21    -2.345794e+00     6.137593e-06
    22    -2.

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [3.6,1.9]
 * Minimizer: [3.9999753743858513,2.0000386175767653]
 * Minimum: -2.345812e+00
 * Iterations: 35
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 72

In [136]:
optimize(prob.f, prob.g!, prob.initial_x, Optim.Options(show_trace=true))

Iter     Function value   Gradient norm 
     0    -2.134718e+00     8.984647e-01
     1    -2.309688e+00     4.262927e-01
     2    -2.345511e+00     2.931502e-02
     3    -2.345811e+00     7.424241e-04
     4    -2.345812e+00     1.379185e-05
     5    -2.345812e+00     3.411041e-11


Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [3.6,1.9]
 * Minimizer: [3.9999999999894973,2.0000000000192895]
 * Minimum: -2.345812e+00
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 7.57e-06 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 2.68e-11 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 3.41e-11 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 14
 * Gradient Calls: 14

In [137]:
optimize(prob.f, prob.initial_x, Optim.Options(show_trace=true, extended_trace=true))

Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------
     0    -2.134718e+00     2.877686e+00
 * step_type: initial
 * centroid: [3.6, 2.3875]
     1    -2.134718e+00     4.391230e-01
 * step_type: outside contraction
 * centroid: [3.6, 2.3875]
     2    -2.134718e+00     1.897094e-01
 * step_type: outside contraction
 * centroid: [3.6, 2.3875]
     3    -2.302859e+00     7.395993e-02
 * step_type: outside contraction
 * centroid: [3.82813, 2.08281]
     4    -2.302859e+00     1.505622e-02
 * step_type: inside contraction
 * centroid: [3.99922, 1.97617]
     5    -2.302859e+00     1.663197e-02
 * step_type: inside contraction
 * centroid: [3.99922, 1.97617]
     6    -2.317432e+00     1.138520e-02
 * step_type: inside contraction
 * centroid: [3.96123, 2.10821]
     7    -2.330737e+00     8.651347e-03
 * step_type: inside contraction
 * centroid: [3.90822, 1.90786]
     8    -2.338368e+00     3.126527e-03
 * step_type: reflection
 * centroid: [3.97195, 1

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [3.6,1.9]
 * Minimizer: [3.9999753743858513,2.0000386175767653]
 * Minimum: -2.345812e+00
 * Iterations: 35
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 72

In [138]:
optimize(prob.f, prob.g!, prob.initial_x, Optim.Options(show_trace=true, extended_trace=true))

Iter     Function value   Gradient norm 
     0    -2.134718e+00     8.984647e-01
 * Current step size: 1.0
 * g(x): [-0.898465, -0.112354]
 * x: [3.6, 1.9]
     1    -2.309688e+00     4.262927e-01
 * Current step size: 0.27884890893043096
 * g(x): [-0.426293, -0.0821232]
 * x: [3.85054, 1.93133]
     2    -2.345511e+00     2.931502e-02
 * Current step size: 0.6202998473428792
 * g(x): [-0.029315, -0.0199269]
 * x: [3.99091, 1.98315]
     3    -2.345811e+00     7.424241e-04
 * Current step size: 0.8918051761491198
 * g(x): [-0.000742424, 0.000434042]
 * x: [3.99977, 2.00037]
     4    -2.345812e+00     1.379185e-05
 * Current step size: 0.8175820487538299
 * g(x): [1.37918e-5, 8.8738e-6]
 * x: [4.0, 2.00001]
     5    -2.345812e+00     3.411041e-11
 * Current step size: 1.0000504001051904
 * g(x): [-3.41104e-11, 2.26246e-11]
 * x: [4.0, 2.0]


Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [3.6,1.9]
 * Minimizer: [3.9999999999894973,2.0000000000192895]
 * Minimum: -2.345812e+00
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 7.57e-06 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 2.68e-11 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 3.41e-11 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 14
 * Gradient Calls: 14

In [139]:
optimize(prob.f, prob.g!, prob.h!, prob.initial_x, Optim.Options(show_trace=true,extended_trace=true))

Iter     Function value   Gradient norm 
     0    -2.134718e+00     8.984647e-01
 * g(x): [-0.898465, -0.112354]
 * h(x): [1.33906 -0.0472876; -0.0472876 1.17676]
 * x: [3.6, 1.9]
     1    -2.330768e+00     2.841926e-01
 * g(x): [-0.284193, -0.053166]
 * h(x): [2.74753 -0.00648258; -0.00648258 1.21794]
 * x: [3.9051, 1.9554]
     2    -2.345802e+00     4.821525e-03
 * g(x): [-0.000115673, -0.00482152]
 * h(x): [3.24784 -2.37752e-7; -2.37752e-7 1.17772]
 * x: [3.99996, 1.9959]
     3    -2.345812e+00     2.431023e-07
 * g(x): [2.43102e-7, 7.89661e-8]
 * h(x): [3.24805 -8.18346e-15; -8.18346e-15 1.17291]
 * x: [4.0, 2.0]
     4    -2.345812e+00     1.153938e-14
 * g(x): [1.15394e-14, -6.21725e-15]
 * h(x): [1.80223 -8.18346e-15; -4.54073e-15 1.08301]
 * x: [4.0, 2.0]


Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [3.6,1.9]
 * Minimizer: [4.000000000000003,1.9999999999999947]
 * Minimum: -2.345812e+00
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 7.48e-08 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 1.15e-14 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 11
 * Gradient Calls: 11
 * Hessian Calls: 4

If the trace is stored, use `Optim.trace(res)` to grab the trace

In [140]:
res = optimize(obj, method, Optim.Options(store_trace=true))
res_trace = Optim.trace(res) 

LoadError: [91mMethodError: no method matching optimize(::#obj, ::Optim.#method, ::Optim.Options{Void})[0m
Closest candidates are:
  optimize(::Any, ::Any, ::Any, [91m::AbstractArray[39m; kwargs...) at /home/juser/.julia/v0.6/Optim/src/multivariate/optimize/interface.jl:56
  optimize(::Any, ::Any, ::Any, [91m::AbstractArray[39m, [91m::Optim.Options[39m) at /home/juser/.julia/v0.6/Optim/src/multivariate/optimize/interface.jl:67
  optimize(::Any, ::Any, ::Any, [91m::AbstractArray[39m, [91m::Optim.Optimizer[39m) at /home/juser/.julia/v0.6/Optim/src/multivariate/optimize/interface.jl:76
  ...[39m

## Results grabbing API
As we saw above, there are functions for grabbing information store in the output.

Use `Optim.minimizer(res)` instead of `res.minimizer` to allow deprecations from our side!

They are listed in the documentation, but the names should be somewhat self-explanatory:
- `minimizer`
- `minimum`
- `iterations`
- `trace`
- `g_calls`
- etc

## Univariate optimization
Univariate functions can be optimized with many of the methods from the multivariate toolbox, as long as the argument is passed as a one element array.

```
    optimize(f, [g!], initial_x, [method], [options])
```

In [141]:
# Problem 18 from http://infinity77.net/global_optimization/test_functions_1d.html
uniprob = Optim.UnivariateProblems.examples["Problem13"]

Optim.UnivariateProblems.UnivariateProblem("Problem13", Optim.UnivariateProblems.p13, [0.001, 0.99], [0.707107], [-1.5874])

In [142]:
unif = uniprob.f
plot(unif, uniprob.bounds...)

In [143]:
optimize(x->unif(x[1]), [mean(uniprob.bounds)], AcceleratedGradientDescent())

Results of Optimization Algorithm
 * Algorithm: Accelerated Gradient Descent
 * Starting Point: [0.4955]
 * Minimizer: [0.7071067811783611]
 * Minimum: -1.587401e+00
 * Iterations: 6
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 2.97e-06 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.50e-11 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 21
 * Gradient Calls: 21

Or we could use one of the two specialized methods for univariate optimization. The syntax is

```
optimize(f, lower_bound, upper_bound, [method], [kwargs...])
```

In [18]:
optimize(unif, uniprob.bounds...)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [0.001000, 0.990000]
 * Minimizer: 7.071068e-01
 * Minimum: -1.587401e+00
 * Iterations: 10
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 11

### Taking advantage of Julia's type system
Univariate optimization allows for us to pass a DualNumber and thus use ForwardDiff to calculate the derivative of the minimum as a function of some auxilirary variables. This should be coming for multivariate optimization soon.

Say we need to solve some equation $G(y)=0$ where evaluating $G(y)$ requires us to solve an optimization problem
$$
    \text{minimize}_x~f(x,y)
$$
If we need to evaluate the derivative of $G(y)$ we then need to know the derivative of the minimum of the nested problem.

Consider the function

$$
f(x,y) = x^2+xy^2
$$

with minimizer $x^* = -y^2/4$ and minimum $f^*(y)=-y^4/4$. Then we have that $\frac{df^*}{dy}(y)=-y^3$. For $y=1.1$ this means that $\frac{df^*}{dy}(1.1)=-1.331$.

But we can also use Optim.jl and ForwardDiff to get to this

In [40]:
dualf(x,y) = x[1]^2+x*y[1]^2
dualg(param) = Optim.minimum(optimize(x->dualf(x,param), -1,1))
ForwardDiff.gradient(dualg, [1.1])

1-element Array{Float64,1}:
 -1.331

## Outside of JNLS
Several packages exist in Julia that can be used. Most notably, there's a Julia wrapper for the well-known and high quality NLOpt package called... NLOpt.jl.

NLOpt has many solvers available
 - Global optimization
 - Derivative free (Local)
 - Gradient based (Local)
 - Augmented Lagrangian
 
 Especially the first and the constrained optimizers are interesting to us, given Optim.jl's current coverage.

# Nonlinear Least Squares Optimization

### The problem
We have an objective, but it's a special type of objective in that it has the form
$$
 f(x)=\sum_{i=1}^m r_i^2
$$
where $r_i$ is the i'th *residual* given by
$$
r_i = y_i-m(p, x_i)
$$
where $m$ is the so-called model or curve function, $y_i$ is the function value we fit to and $x_i$ are the variables and $p$ is the parameter vector (notice the change of notation!).

Such a problem is often motivated by a setup where we observe the model with noise.
$$
y_{i}^{obs} = m(p, x_i^{obs})+noise
$$

### JuliaNLSolvers III: LsqFit.jl

### Example: An exponential model
Say we have a model of the form
$$
   m(x, p) = p_1\exp(p_2x)
$$
that is univariate in $x$. Let us try to plot the function on the interval $[0:10]$, and add some randomly generated data.

In [20]:
# x: array of independent variables
# p: array of model parameters
model(x, p) = p[1]*exp.(-x.*p[2])

# some example data
# xdata: independent variables
# ydata: dependent variable
xdata = linspace(0,10,20)
ydata = model(xdata, [1.0 2.0]) + 0.01*randn(length(xdata))

20-element Array{Float64,1}:
  1.00163   
  0.357377  
  0.128355  
  0.0409426 
  0.0190074 
  0.00911234
 -0.0276199 
  0.00231194
  0.00453784
 -0.0203142 
 -0.00170607
 -0.00617254
 -0.0178912 
  0.00711063
  0.00584508
  0.00168497
  0.0117418 
 -0.00946779
 -0.00623248
 -0.00674679

![plot(x->1*exp(-2*x), 0,10, legend=false); scatter!(xdata, ydata);](https://juliabox.com/notebooks/data.png)

### JuliaNLSolvers III: LsqFit.jl
The syntax is very simple. We *need* to provide a model, some data, and a starting value. We *can* provide a Jacobian, lower and upper bounds, and weights.

In [86]:
# Pkg.add("LsqFit")
using LsqFit
p0 = [0.5, 0.5]
fit = curve_fit(model, xdata, ydata, p0)

LsqFit.LsqFitResult{Float64,1}(18, [1.00191, 1.96289], [0.000282523, -0.000796079, -0.00144698, 0.00422409, -0.00293248, -0.00339125, 0.029656, -0.00158727, -0.00427993, 0.020406, 0.00173874, 0.00618416, 0.0178953, -0.00710916, -0.00584456, -0.00168478, -0.0117417, 0.00946781, 0.00623249, 0.00674679], [1.0 0.0; 0.355902 -0.187674; … ; 8.39346e-9 -7.96687e-8; 2.98722e-9 -2.99294e-8], true, Float64[])

![](error.png)

### Example: waves, waves, waves (continued)

Earlier we looked a function similar to
$$ 
f(t) = sin(5.1*x)/x 
$$

Imagine we observe such a signal (or more) where t is time from time 0. The problem is, our measuring equipment fired off at irregular intervals and it's rather cheap so the recorded signals are noisy.

In [ ]:
model(x, p) = sin.(p[1]*x)./x
xdata = rand(10)*10. # random uniform from 0 to 10
ydata = model(xdata, [5.1,]) + 0.01*randn(length(xdata))


fit = curve_fit(model, xdata, ydata, [0.3,])

LsqFit.LsqFitResult{Float64,1}(9, [0.380287], [-0.0164868, 0.0479606, 0.0170075, -0.0331252, 0.0438394, -0.0606297, 0.020777, -0.0364514, 0.0242265, -0.0025678], [-0.929686; -0.902402; … ; -0.362437; 0.463296], true, Float64[])

In [1]:
plot((0.0:0.01:10.0),  x->model(x, fit.param), label= "estimated curve")
plot!((0.0:0.01:10.0),  x->model(x, [5.1,]), label= "true curve")
scatter!(xdata, ydata, label="data")

LoadError: [91mUndefVarError: plot not defined[39m

# Solving Systems of Equations

Consider the following problem of finding an $x$ such that
$$
\begin{align}
f_1(x) &= 0\\
f_2(x) &= 0\\
&\vdots\\
f_N(x) &= 0
\end{align}
$$
where $x$ is in some subset of $\mathbb{R}^N$. We write $F(x)=0$ for short. 

## Example: fixed points
A special type of problems are fixed point problems
$$
F(x) = x
$$
but these can obviously be solved by the same methods by setting $\tilde{F}(x)=F(x)-x$ to zero.

We are going to use methods much like above, but there is no objective this time! We just need a zero/solution.

Without going into too much details, you can think of these methods much like the second order methods from above, although we have no "objective function". At each iteration, we choose a direction

$$
s = J(x)^{-1}F(x)
$$

where $J(x)$ is the Jacobian of $F(x)$, and then we either choose a step size based on a line search (by use of a merit function) or a trust region approach.

Syntax is straight forward

```
   nlsolve(F, [J], initial_x; kwargs...)
```
where F fills a vector with the results of the $f_i$'s and $J$ fills the associated Jacobian

### Example: Rosenbrock FOCs as a system of nonlinear equation
Let's try solving a system of equations that happen to have the same form as the FOCs for minimizing the Rosenbrock function.

In [54]:
# Pkg.add("NLsolve")
using NLsolve
prob = Optim.UnconstrainedProblems.examples["Rosenbrock"]
nlsolve((y,x)-> prob.g!(x,y), prob.initial_x) # order of arguments...

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.0, 0.0]
 * Zero: [1.0, 1.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 64
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 65
 * Jacobian Calls (df/dx): 47

### Example: Butt-cheek function

Consider the following system of equations
$$
    \begin{align}
    x &= 0\\
    y(y^2-1)&=0
    \end{align}
$$
The first condition is trivial. How does the second one look?

In [53]:
plot(y->y*(y^2-1),-2,2)

In [85]:
function bf!(x, fvec)
    fvec[1] = x[1]
    fvec[2] = x[2]*(x[2]^2-1)
end
nlsolve(bf, [0.0, -2.4])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.0, -2.4]
 * Zero: [0.0, -1.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 7
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 8
 * Jacobian Calls (df/dx): 8

We can also supply the Jacobian and the combined F and Jacobian manually, as to avoid finite differencing.

In [131]:
function bg!(x, gvec)
    gvec[1,1] = 1.0
    gvec[1,2] = 0.0
    gvec[2,1] = 0.0
    gvec[2,2] = 3x[2]^2-1
end
function bfg!(x, fvec, gvec)
    fvec[1] = x[1]
    x2squared = x[2]^2
    fvec[2] = x[2]*(x2squared-1)
    
    gvec[1,1] = 1.0
    gvec[1,2] = 0.0
    gvec[2,1] = 0.0
    gvec[2,2] = 3x2squared-1
    f
end
df = DifferentiableMultivariateFunction(bf!, bg!, bfg!)
nlsolve(df, [0.0, -2.4])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.0, -2.4]
 * Zero: [0.0, -1.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 7
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 8
 * Jacobian Calls (df/dx): 8

## Two methods
- Newton + trust region (default method)

```julia
    optimize(f, [g!, fg!], initial_x, method = :trust_region)
```
- Newton + linesearch (defaults to no linesearch)

```julia
    optimize(f, [g!, fg!], initial_x, method = :newton)
```

Not as finished as Optim, but there are several PRs currently active, that will improve the package significantly. We also need to unify interaces.

# Exercises

## Exercise: Hosaki (lessons from the workshop)

Plot the "Hosaki" problem from Optim.UnconstrainedProblems on $x\in[0,5]x[0,5]$

Initialize some different solvers of your own choice at the initial point given by the `initial_x` field of the problem instance. Do they converge to the same minimum?

Initialize the same solvers at x=[2.0,1.9], x=[2.0, 2.9], and one with a slight pertubation to x[1] (say x=[1.999999, 1.9] or x=[2.000001,1.9]). Can you explain the behavior given the plot from above?

Exercise: LineSearches

Based on the examples in UnconstrainedExamples, try to compare different line search choices. One possibility is to create a table the number of f, g!, and if applicable h! calls, time elapsed, and
final function value

## Exercise: LsqFit

Try generating data according from a model+noise setup as discussed above for a model of your own choice, and see if you can recover parameters.

Does changing the number of observations affect your results? Does it affect the estimated standard errors from `estimate_errors`?

## Exercise: NLsolve

Try finding a solution to the system of equations given by 
$$
\nabla f = 0
$$
where $\nabla f$ is the gradient from the Hosaki problem we looked at in the exercise above (note: remember the argument order)

By varying the initial point, are you able to recover the critical points we found earlier?